**Referenece**: <https://adv-r.hadley.nz/r6.html>

In [2]:
library(R6)

# Classes and Methods

In [5]:
normal <- R6Class(
  "normal",
  list(
    # to initialize attributes for class
    # you have to define it outside here
    # if omit `mu` and `sig` here, when calling initialize will raise error locking environment
      
    # define public attributes for class `normal`
    mu = 0,
    sig = 0,
    # __init__
    initialize = function(mu, sig) {
      self$mu <- mu
      self$sig <- sig
    },
    # sample n values 
    rvs = function(n) rnorm(n, self$mu, self$sig),
    # magic function __print__
    print = function(...) {
      cat("A normal distribution with", self$mu, "mean and", self$sig, "standard deviation")
      # side-effect, return self invisibly for method chaining
      invisible(self)
    }
  )
)

# create a new instance of a class with `class$new`
mod <- normal$new(10, 3)
# print mod
mod

mod$rvs(10)



A normal distribution with 10 mean and 3 standard deviation

[1]  6.898218  8.089823 10.659933  8.581555 13.649080 11.271515  8.142396
 [8] 11.661738 10.518513  9.575810

In [6]:
# adding new methods and attributes for class normal
normal$set("public", "cdf", function(value) pnorm(value, self$mu, self$sig))
normal$set("public", "name", "normal distribution")
           


mod <- normal$new(10, 3)

mod$name           
           
mod$rvs(10)

mod$cdf(10)

[1] "normal distribution"

[1]  2.848392 10.969869 14.390847 11.231909 12.694075 10.528594  7.892913
 [8] 11.961315  8.097062 10.286746

[1] 0.5

## Inheritance

In [12]:
# inheritance
StandardNormal <- R6Class('StandardNormal', 
                          list(mu = 0, 
                               sig = 1,
                               # access parent with `super`
                               initialize = function() super$initialize(0, 1),
                               print = function() {
                                   cat('Standard Normal is a special case of normal distribution\n')
                                   super$print()
                               }),
                          # set the class to inherit from
                          inherit = normal)

my_standard_normal <- StandardNormal$new()

my_standard_normal

my_standard_normal$rvs(5)

my_standard_normal$cdf(0)

Standard Normal is a special case of normal distribution
A normal distribution with 0 mean and 1 standard deviation

[1]  0.4041413 -0.4326019 -2.2496547 -0.7458290 -0.2999957

[1] 0.5

**NOTES**: subclass cannot access private attributes of parent class, but it can access private method of parent class

In [122]:
A <- R6Class(
  classname = "A",
  private = list(
    field = "foo",
    method = function() {
      "bar"
    }
  )
)

B <- R6Class(
  classname = "B",
  inherit = A,
  public = list(
    test = function() {
      cat("Field:  ", super$field, "\n", sep = "")
      cat("Method: ", super$method(), "\n", sep = "")
    }
  )
)

B$new()$test()

Field:  
Method: bar


# Controlling access: public, private and active

> control via `public`, `private` and `active` keyword argument of function **`R6Class`**

## private

In [18]:
Pokemon <- R6Class('Pokemon', 
                  list(
                      # define public attributes
                      name = NULL,
                      initialize = function(name, price) {
                          self$name <- name
                          # self.__price__
                          private$price <- price
                      },
                      print = function(...) cat('my price is: ', private$price, '$')
                  ),
                  # define pivate attributes
                  private = list(price = NULL))

pikachu <- Pokemon$new('Pikachu', 999)
pikachu

my price is:  999 $

In [19]:
# you cannot access private attributes and methods
pikachu$price

NULL

In [20]:
# but you can access public attributes and methods
pikachu$name

[1] "Pikachu"

## active (getter and setter)

In [79]:
Clipper <- R6Class("Clipper",
  public = list(
    lower = NULL,
    upper = NULL,
    initialize = function(initial_value, lower, upper) {
      self$lower <- lower
      self$upper <- upper
      self$value <- initial_value
    },
    # increase current value by `amount`
    inc = function(amount) {
      self$value <- self$value + amount
      # this function is side-effect, so return invisibly
      invisible(self)
    },
    desc = function(amount) {
      self$value <- self$value - amount
      # this function is side-effect, so return invisibly
      invisible(self)
    }
  ),
  active = list(
    # a setter that clip value between `lower` and `upper`
    value = function(x) {
      # if we pass no arugment, it means we are trying to access value, so --> getter
      if (missing(x)) {
        return(private$val)
      }
      # setter
      private$val <- min(max(x, self$lower), self$upper)
      # return invisibly
      invisible(self)
    }
  ),
  private = list(
    val = NULL
  )
)

my_clipper <- Clipper$new(25, lower = 0, upper = 10)


In [72]:
# getter
my_clipper$value

[1] 10

In [73]:
# set value to negative, setter clip it to 0
my_clipper$value <- -10
my_clipper$value

[1] 0

In [74]:
my_clipper$value <- 5
my_clipper$value

[1] 5

In [75]:
# increase current value by 3
my_clipper$inc(3)
# get the value of the current value
my_clipper$value

[1] 8

In [76]:
# increase the current value by 100, because of setter, it will be clipped to return 10, not 108
my_clipper$inc(100)
my_clipper$value

[1] 10

In [77]:
# decrease the current value by 3
my_clipper$desc(3)
my_clipper$value


[1] 7

# Reference sematics

The big difference of R6, it is not copied on modified (like instances of class in python)

In [89]:
clipper1 <- Clipper$new(5, 0, 10)
clipper2 <- clipper1

clipper1$value
clipper2$value

[1] 5

[1] 5

In [90]:
# change the value of clipper1 will also change the value of clipper2
clipper1$value <- 11

clipper1$value
clipper2$value

[1] 10

[1] 10

Explicitly copy with `$clone`

In [91]:
clipper3 <- clipper1$clone()

# change the value of clipper 3 will not affect the value of clipper 1
clipper3$value <- 1

clipper3$value
clipper1$value

[1] 1

[1] 10

> NOTE: `clone` does not recursively copy deeply nested R6. use `clone(deep = TRUE)` to do this

## 1. Reasoning

Read the chapter

## 2. Finalizers

The clean up when the instance is deleted. Like **`__del__`** in python

In [98]:
TemporaryFile <- R6Class("TemporaryFile", list(
  path = NULL,
  initialize = function() {
    # open file
    self$path <- tempfile()
  },
  # handle function when instance is deleted
  finalize = function() {
    message("Cleaning up ", self$path)
    # close file
    unlink(self$path)
  }
))

my_file <- TemporaryFile$new()
my_file$path

[1] "/tmp/RtmpWah1N8/fileb0422351ab24"

In [99]:
rm(my_file)

# Exercises

> 1.Create an R6 class that represents a shuffled deck of cards. You should be able to draw cards from the deck with `$draw(n)`, and return all cards to the deck and reshuffle with `$reshuffle()`. 

In [106]:
Deck <- R6Class(
  "Deck",
  list(
    # define public attributes
    deck = NULL,
    initialize = function() {
      suit <- c("H", "D", "S", "C")
      value <- c("A", 2:10, "J", "Q", "K")
      cards <- paste0(rep(value, 4), suit)
      # store all cards in a deck privately
      private$cards <- cards
      # store the current state of the deck
      self$deck <- cards
    },
    draw = function(n) {
      ncards <- length(self$deck)
      # raise error when there are not enough cards to draw
      if (n > ncards) {
        stop("You have only ", ncards, " cards left but you draw ", n, " cards", call. = FALSE)
      }
      # randomly draw n cards from the deck
      ix <- sample(1:ncards, size = n)
      result <- self$deck[ix]
      # remove drawed cards from the deck
      self$deck <- self$deck[-ix]
      result
    },
    reshuffle = function() {
      self$deck <- sample(private$cards, 52)
      invisible(self)
    }
  ),
  # define private attributes
  private = list(cards = NULL)
)

deck_of_cards <- Deck$new()

deck_of_cards$reshuffle()

deck_of_cards$deck

[1] "KH"  "QD"  "9H"  "6H"  "KD"  "4H"  "9D"  "4C"  "5H"  "6D"  "4S"  "QC" 
[13] "AS"  "6S"  "JS"  "QH"  "2D"  "3H"  "AC"  "AH"  "5S"  "10D" "6C"  "5D" 
[25] "KS"  "2H"  "JD"  "JC"  "9C"  "QS"  "KC"  "2C"  "10C" "5C"  "10S" "8D" 
[37] "AD"  "3C"  "9S"  "8H"  "3S"  "4D"  "10H" "7S"  "JH"  "7H"  "8C"  "8S" 
[49] "7C"  "3D"  "7D"  "2S"

In [107]:
deck_of_cards$draw(5)

[1] "10D" "AD"  "KS"  "5S"  "5D"

In [108]:
deck_of_cards$draw(45)

[1] "4H"  "QC"  "9D"  "KD"  "7D"  "6S"  "7S"  "8H"  "KH"  "AS"  "10C" "AH" 
[13] "5C"  "4D"  "3H"  "QS"  "6C"  "4C"  "8D"  "JS"  "7C"  "8S"  "3D"  "9S" 
[25] "10S" "6H"  "2D"  "9C"  "2H"  "JC"  "QD"  "2C"  "3C"  "8C"  "2S"  "KC" 
[37] "6D"  "JH"  "4S"  "7H"  "AC"  "JD"  "3S"  "10H" "5H"

In [109]:
try(deck_of_cards$draw(10))

Error : You have only 2 cards left but you draw 10 cards


In [110]:
deck_of_cards$reshuffle()
deck_of_cards$deck

[1] "AC"  "6C"  "7C"  "QC"  "2S"  "KD"  "5S"  "8S"  "6H"  "9D"  "JC"  "2H" 
[13] "10S" "7H"  "3S"  "9H"  "7S"  "JS"  "4H"  "5H"  "AH"  "4C"  "JD"  "JH" 
[25] "9C"  "7D"  "8D"  "10C" "QD"  "QH"  "5D"  "10H" "QS"  "2C"  "4D"  "5C" 
[37] "6D"  "AD"  "3D"  "KH"  "6S"  "8C"  "8H"  "KS"  "3C"  "AS"  "10D" "3H" 
[49] "9S"  "2D"  "KC"  "4S"

---
>2. Create an R6 class that allows you to get and set the current time zone. When setting the time zone, make sure the new time zone is in the list provided by `OlsonNames()`.

In [113]:
Timezone <- R6Class("Timezone",
  private = list(
    tz = NULL
  ),
  public = list(
    initialize = function() {
      # getter and setter
      self$timezone <- Sys.timezone()
    }
  ),
  active = list(
    timezone = function(tz) {
      if (missing(tz)) {
        return(private$tz)
      }
      if (!tz %in% OlsonNames()) {
        stop(tz, " is an invalid timezone")
      }
      private$tz <- tz
      invisible(self)
    }
  )
)

my_tz <- Timezone$new()

my_tz$timezone

my_tz$timezone <- 'Europe/London'

my_tz$timezone

[1] "Asia/Ho_Chi_Minh"

[1] "Europe/London"

In [112]:
try(my_tz$timezone <- 'Hai duong')

Error in (function (tz)  : Hai duong is an invalid timezone


---
>3. Create a bank account R6 class that stores a balance and allows you to deposit and withdraw money. Create a subclass that throws an error if you attempt to go into overdraft.

In [116]:
BankAccount <- R6Class(
  "BankAccount",
  list(
    balance = 0,
    deposit = function(amount = 0) {
      stopifnot(is.numeric(amount))
      self$balance <- self$balance + amount
      invisible(self)
    },
    withdraw = function(amount = 0) {
      stopifnot(is.numeric(amount))
      self$balance <- self$balance - amount
      invisible(self)
    }
  )
)

account <- BankAccount$new()

account$balance

account$deposit(100)
account$balance

account$withdraw(200)
account$balance

[1] 0

[1] 100

[1] -100

In [118]:
BankAccountStrict <- R6Class(
  "BankAccountStrict",
  inherit = BankAccount,
  public = list(
    withdraw = function(amount) {
      stopifnot(is.numeric(amount))
      if (self$balance < amount) {
        stop("You do not have enough money to withdraw", call. = FALSE)
      }
      super$withdraw(amount)
    }
  )
)

account_strict <- BankAccountStrict$new()

account_strict$deposit(100)

account_strict$balance

try(account_strict$withdraw(200))

[1] 100

Error : You do not have enough money to withdraw


>Create a class that allows you to write a line to a specified file. You should open a connection to the file in `$initialize()`, append a line using cat() in `$append_line()`, and close the connection in `$finalize()`.

In [119]:
FileWriter <- R6Class(
  "FileWriter",
  list(
    con = NULL,
    initialize = function(filename) {
      self$con <- file(filename, open = "a")
    },
    finalize = function() {
      print('Closing...')
      close(self$con)
    },
    append_line = function(txt) {
      cat(txt, "\n", file = self$con)
    }
  )
)

tmp_file <- tempfile()
my_file <- FileWriter$new(tmp_file)
my_file$append_line('I love you, Pika Pika!')
readLines(tmp_file)

rm(tmp_file)


[1] "I love you, Pika Pika! "

> 4. Create a bank account class that prevents you from directly setting the account balance, but you can still withdraw from and deposit to. Throw an error if you attempt to go into overdraft.

In [121]:
BankAccountSecurity <- R6Class("BankAccountSecurity",
  private = list(
    balance = 0
  ),
  public = list(
    get_balance = function() {
      private$balance
    },
    deposit = function(amount = 0) {
      stopifnot(is.numeric(amount))
      private$balance <- private$balance + amount
      invisible(self)
    },
    withdraw = function(amount = 0) {
      stopifnot(is.numeric(amount))
      if (private$balance < amount) {
        stop("You do not have enough money to withdraw", call. = FALSE)
      }
      private$balance <- private$balance - amount
      invisible(self)
    }
  )
)

my_account <- BankAccountSecurity$new()

my_account$deposit(100)

my_account$get_balance()

try(my_account$withdraw(1000))


[1] 100

Error : You do not have enough money to withdraw
